03. DATA PREPARATION

Ball Preparation

In [1]:
import pandas as pd
import numpy as np

balldf = pd.read_csv('../../data/raw-coords/ball-3-fullvideo.csv')

#choose best coords if more than one
# see what frames have more than one ball
multiple_balls = balldf[balldf['balls'].astype(str).str.contains(';', na=False)]
print(multiple_balls[['frame_index', 'balls']])

     frame_index                                              balls
98            98                    7497.87,2274.17;4250.71,1043.92
99            99                    7676.05,2173.04;2730.72,1950.40
102          102                    8386.67,2691.40;6296.66,3195.41
106          106                    8961.26,3074.04;9290.05,3264.22
108          108    9154.89,3164.93;9365.34,3259.58;7440.86,4986.66
114          114                     9795.57,3603.52;4714.25,934.54
131          131                   10600.56,3130.46;5440.80,2280.61
136          136                   9221.64,1331.83;10026.16,3649.06
138          138                    8658.38,1258.53;9944.20,3699.75
196          196                   10583.05,1353.97;9789.05,3247.78
197          197  11000.08,911.55;10206.36,1201.70;11388.31,3469.20
204          204                  11538.79,2302.03;11471.00,3716.57
212          212                    8937.58,3502.83;9030.92,3569.17
216          216                  11814.37,1504.

In [ ]:
def choose_best_ball_by_interpolation(row_idx, balls_str, df, col='balls'):
    """
    Escolhe a coordenada de bola mais próxima da interpolação linear entre as anteriores e posteriores.
    Se não houver anterior ou posterior, retorna a primeira.
    """
    if pd.isna(balls_str) or ';' not in balls_str:
        return balls_str  # já está ok ou vazio

    # Extrai as opções de bolas
    balls = [b.strip() for b in balls_str.split(';')]
    balls_xy = []
    for b in balls:
        try:
            x, y = map(float, b.split(','))
            balls_xy.append((x, y))
        except Exception:
            continue

    # Busca anterior
    prev_idx = row_idx - 1
    while prev_idx >= 0 and (pd.isna(df.at[prev_idx, col]) or ';' in str(df.at[prev_idx, col])):
        prev_idx -= 1
    prev_xy = None
    if prev_idx >= 0 and pd.notna(df.at[prev_idx, col]) and ';' not in str(df.at[prev_idx, col]):
        try:
            prev_xy = tuple(map(float, str(df.at[prev_idx, col]).split(',')))
        except Exception:
            prev_xy = None

    # Busca posterior
    next_idx = row_idx + 1
    while next_idx < len(df) and (pd.isna(df.at[next_idx, col]) or ';' in str(df.at[next_idx, col])):
        next_idx += 1
    next_xy = None
    if next_idx < len(df) and pd.notna(df.at[next_idx, col]) and ';' not in str(df.at[next_idx, col]):
        try:
            next_xy = tuple(map(float, str(df.at[next_idx, col]).split(',')))
        except Exception:
            next_xy = None

    # Se não tem anterior ou posterior, retorna a primeira opção
    if prev_xy is None or next_xy is None:
        return balls[0]

    # Calcula interpolação linear
    t = (row_idx - prev_idx) / (next_idx - prev_idx)
    interp_x = prev_xy[0] + (next_xy[0] - prev_xy[0]) * t
    interp_y = prev_xy[1] + (next_xy[1] - prev_xy[1]) * t

    # Escolhe a bola mais próxima da interpolação
    best_ball = min(balls_xy, key=lambda xy: (xy[0] - interp_x) ** 2 + (xy[1] - interp_y) ** 2)
    return f"{best_ball[0]},{best_ball[1]}"

# Exemplo de uso para corrigir todas as linhas com múltiplas bolas:
def fix_multiple_balls(df, col='balls'):
    df = df.copy()
    for idx, row in df.iterrows():
        if pd.notna(row[col]) and ';' in str(row[col]):
            df.at[idx, col] = choose_best_ball_by_interpolation(idx, row[col], df, col=col)
    return df

# Aplicando na sua tabela balldf:
balldf = fix_multiple_balls(balldf, col='balls')

# see what frames have more than one ball
multiple_balls = balldf[balldf['balls'].astype(str).str.contains(';', na=False)]
print(multiple_balls[['frame_index', 'balls']])

#save the fixed dataframe
balldf.to_csv('../../data/raw-coords/ball-3-fullvideo-fixed.csv', index=False)

In [ ]:
filepath = '../../data/raw-coords/3-fullvideo.csv'


df = pd.read_csv(filepath)

traindf = df

In [ ]:
#drop empty columns
traindf = df.dropna(axis=1, how='all')

See Transitions of ids or detect ids of referees or other entities - manually

Team 0 - PSG

In [ ]:
#TEAM 0 - PSG 
#gk - id281_0 ff106 lf127 -> id268_1 lf130 -> 323_0 lf137 -> 281_0 lf181 -> 428_0 lf152 -> 524_0 ff-202 lf-230 -> 614_0 ff-233 lf-290 -> 725_0 ff-292 lf-401 -> 1074_1 ff-428 lf-433
#lb - id5_0 ff0 lf67 -> id129_0 ff69 lf92 -> id188_0 ff94 lf103 -> id270_0 ff105 lf133 -> id_365_0 ff138 lf194 -> id502_0 ff196 lf209 -> id537_0 ff212 lf230 ->id608_0 ff232 lf250 -> id639_0 ff252 lf263 -> id652_0 ff265 lf285 -> id652_0 ff287 lf305-309? (ver este frame pois o outro jogar fica com este id pq passaram "um por cima do outro") -> id765_0 ff310 lf392 -> id855_0 ff394 lf428 -> id1104_0 ff431 lf438 ->id1027_0 ff439 (falha no frame 443) lf451 -> id582_1 ff452 lf452 -> id1104_0 ff453 lf454 -> id1212_0 ff458 lf470 -> id1339_0 ff472 lf503 -> id1420_0 ff505 lf505 -> id1027_0 ff509 lf544
#cb1 - id13_0 ff0 lf97  -> id255_0 ff103 lf132 -> id381_0 ff140 lf203 -> id528_0 ff205 lf211 -> id539_0 ff213 (216-219 fica noutra equipa) lf230 ->id381_0 ff232 lf264 -> id660_0 ff266 lf272 -> id381_0 ff273 lf290 -> id660_0 ff291 lf358 -> id809_0 ff373 lf408 (376-380 e 401  jogadores sobrepostos fica no da outra equipa) -> id869_1 ff409 lf419 -> id1028_0 ff424 lf428 -> id1094_0 ff430 lf452 -> id1252_0 ff460 lf470 -> id1341_0 l/ff472 -> id1349_0 ff474 lf479 -> id1379 ff481 lf482 -> id1392_0 l/ff485 -> id1396_0 l/ff488 -> id1402_0 ff491 lf494 -> id1396 ff496 -> id1379_0 ff496 lf499 -> id1402_0 ff503 lf507 -> id1448_0 ff509 lf517 -> id1402_0 ff518 lf544
#cb2 - id10_0 ff0 lf60 -> id99_0 ff63 lf68 -> id10 ff69 lf84 -> id70_1 ff 85 lf86 -> id172_0 ff88 lf95 -> id197_0 l/ff97 -> id226_0  l/ff100 -> id235_0 l/ff101 -> id247_0 ff103 lf132 (existem frames pelo meio que ele não apanha: ex 105) -> id387_0 ff141 lf150 -> id433_0 l/ff154 -> id437_0 ff156 lf173 -> id468_0 ff175 lf194 -> id503 ff197 lf220 -> id581_0 ff224 lf226 (226-228 jog equipa contraria fica com id) -> id604_0 ff229 lf296 -> id368_0 ff297 lf303 -> id604_0 ff304 lf376 -> id822_0 ff378 lf402 -> id876_0 l/ff404 -> id604_0 ff405 lf410 -> id931_0 ff414 lf418 (troca para jogador sobreposto) -> id1063_0 ff427 lf451 -> id1263_0 l/ff461 -> id1243_1 ff466 lf471 -> id1346_0 ff473 lf478 -> id1135_1 ff479 -> id1378_0 ff481 lf482 -> id1220_1 ff483 lf506 -> id1346_0 ff508 lf509 -> id1341_0 ff510 lf544 
#rb -  id19_0 ff0 lf40 - id45_0 ff42 lf67 -> id130_0 ff69 lf76 -> id146_0 ff78 lf91 -> id186_0 ff94 lf97 -> id131 ff98 -> id45_0 ff99 ->id254_0 ff103 lf132 -> id393_0 ff142 lf148 -> id368_0 (id jogador outro msm equipa) ff150 lf216 -> id584 ff224 lf262 -> id661_0 ff266 lf284 -> id739_1 ff296 lf412 (este id está em constante troca entre 2 jogadores) -> id822 ff413 lf414 -> id763 ff415 lf416 -> id876 ff417 -> id604 ff419 -> id978_1 ff424 -> id931_0 ff425 -> id1083_0 ff429 lf453 -> id1093_1 ff454 -> id1268_0 ff461 -> id1296_0 ff464 -> id1309_0 ff466 lf1309 -> id1373 ff478 lf479 -> id1309 ff480 lf481 -> id1386_0 ff484 lf503 -> id1425 ff506 lf507 -> id1373_0 ff508 lf 509 -> id1372_1 ff510 lf517 -> id1425_0 f518 lf519 -> id1373_0 ff521 lf544
#cm1 (vitinha) - id2_0 ff0 lf60 -> id108_0 ff64 lf97 -> id105 ff100 -> id256_0 ff103 lf132  -> id72_0 ff133 lf137 -> id367_0 ff138 lf143 -> id422_0 ff148 lf156 -> id392_1 ff157 lf158 -> id443_0 ff160 lf169 -> id474_0 ff178 -> id477_0 ff180 lf194  -> id468_0 ff198 -> id518_0 ff200 lf220 -> id580_0 ff224 -> id592_0 ff226 lf238 -> id581_0 ff240 -> id626_0 ff242 lf264 -> id657_0 ff266 lf270 -> id626_0 ff271 lf279 -> id657_0 ff280 lf281 -> id692_0 ff283 -> id700_0 ff286 -> id724_0 ff292 lf420 (334 e 335 para e volta no 336) -> id1027_0 ff424 lf429 -> id1103 ff431 lf452 -> id1249_0 ff460 lf492 (frame 469-471 para e volta) -> id1412_0 ff495 lf519 -> id1489_0 ff521 lf522 -> id1445_1 ff523  lf524 -> id1499_1 ff525 -> id1503_1 ff530 -> id1489_0 ff532 lf539 -> id1511_0 ff541 lf542 ->id1445_1 ff543 lf544
#cm2 (fabian ruiz) - id4_0 ff0 lf52 -> id63_0 ff55 -> id66_0 ff57 lf76 -> id147_0 ff78 lf97 (muda para jogador adver sobreposto) -> id264_0 ff103 lf104 -> id279_0 ff106 lf113 -> id264_0 (mudou de jogar adv sobreposto) ff116 lf121 -> id248_1 ff122 -> id306_0 ff125 lf217 -> id556_0 ff223 lf234 -> id393_0 ff235 -> id556_0 ff236 lf258 -> id544_1 ff259 lf261 -> id552_0 ff262 -> id671_0 ff269 lf302 -> id652_0 (ficou com o do comp de equipa) ff304 lf428 ->id1089_0 ff430 -> id582_1 ff432 -> id1116_0 ff434 lf453 -> id1238_0 ff459 lf530 -> id1507_0 ff534 lf543
#cm3 (joao neves) - id16_0 ff0 lf10 -> id37_0 ff28 lf50 -> id61_0 ff55 lf67 -> id131_0 ff70 lf75 -> id61 ff76 lf77 -> id160_0 ff81 lf84 -> id45_0 ff89 -> id183_0 ff92 lf97 -> id257_0 ff103 lf127 -> id276_1 ff128 lf135 -> id368_0 ff138 lf149 (mudou para um comp de equipa) -> id434_0 ff154 lf163 -> id432_1 ff164 lf182 -> id486_0 ff184 -> id393_0 ff186 lf217 -> id432_1 ff218 -> id536_0 ff222 lf224 -> id597_0 ff227 lf263 -> id664_0 ff266 lf304 -> id760_0 ff307 lf401 -> id625_0 ff402 lf419 -> id984_0 ff421 -> id1011_0 ff423 lf429 -> id836_0 ff430 lf431 -> id1111_0 ff433 lf452 -> id1216_0 ff458 lf469 -> id1111_0 ff470 -> id1343_0 ff472 -> id1106_1 ff473 -> id1250_1 ff474 lf477 -> id1389_0 ff484 lf502 -> id1250_1 ff508 -> id1347_0 ff513 -> id1494_0 ff522 -> id1502_0 ff526 lf544                           
#st(dembele) - id17_0 ff0 lf58 -> id125_0 ff66 lf88 -> id17_0 ff89 lf91 -> id125_0 ff92 lf203 -> id543_0 ff216 lf219 -> id248_1 ff222 lf230 -> id607_0 ff232 lf422 -> id1073_0 ff428 lf456 -> id1310_0 ff466 -> id1320_0 ff468 lf473 -> id1376_0 ff480 -> id1383_0 ff482 -> id1310_0 ff487 lf521 -> id1495_0 ff523 lf544
#lw (kvara) - id1_0 ff0 lf68 -> id 132_0 ff70 -> id135_0 ff72 lf88 -> id176_0 ff90 lf98 -> id253_0 ff103 lf104 -> id275_0 ff106 lf128 -> id111_1 ff129 lf142 -> id404_0 ff144 lf217 -> id552_0 ff219 lf232 -> id596_1 ff235 lf243 -> id635_1 ff251 lf253 -> id544_1 ff254 lf255 -> id644_0 ff257 lf264 -> id596_1 ff265 lf281 -> id544_1 ff282 lf284 -> id656_1 ff285 lf286 -> id708_1 ff298 lf396 -> id859_0 ff398 lf411 ff416 lf429 -> id1101_0 ff431 lf451 -> id1306_0 ff466 lf544

In [ ]:
# CB1
df.loc[103:, 'id_13_team_0'] = df.loc[103:, 'id_255_team_0']
df.loc[140:, 'id_13_team_0'] = df.loc[140:, 'id_381_team_0']
df.loc[205:, 'id_13_team_0'] = df.loc[205:, 'id_528_team_0']
df.loc[213:, 'id_13_team_0'] = df.loc[213:, 'id_539_team_0']
df.loc[232:, 'id_13_team_0'] = df.loc[232:, 'id_381_team_0']
df.loc[266:, 'id_13_team_0'] = df.loc[266:, 'id_660_team_0']
df.loc[273:, 'id_13_team_0'] = df.loc[273:, 'id_381_team_0']
df.loc[291:, 'id_13_team_0'] = df.loc[291:, 'id_660_team_0']
df.loc[373:, 'id_13_team_0'] = df.loc[373:, 'id_809_team_0']
df.loc[409:, 'id_13_team_0'] = df.loc[409:, 'id_869_team_1']
df.loc[424:, 'id_13_team_0'] = df.loc[424:, 'id_1028_team_0']
df.loc[430:, 'id_13_team_0'] = df.loc[430:, 'id_1094_team_0']
df.loc[460:, 'id_13_team_0'] = df.loc[460:, 'id_1252_team_0']
df.loc[472:, 'id_13_team_0'] = df.loc[472:, 'id_1341_team_0']
df.loc[474:, 'id_13_team_0'] = df.loc[474:, 'id_1349_team_0']
df.loc[481:, 'id_13_team_0'] = df.loc[481:, 'id_1379_team_0']
df.loc[485:, 'id_13_team_0'] = df.loc[485:, 'id_1392_team_0']
df.loc[488:, 'id_13_team_0'] = df.loc[488:, 'id_1396_team_0']
df.loc[491:, 'id_13_team_0'] = df.loc[491:, 'id_1402_team_0']
df.loc[496:, 'id_13_team_0'] = df.loc[496:, 'id_1396_team_0']
df.loc[496:, 'id_13_team_0'] = df.loc[496:, 'id_1379_team_0']
df.loc[503:, 'id_13_team_0'] = df.loc[503:, 'id_1402_team_0']
df.loc[509:, 'id_13_team_0'] = df.loc[509:, 'id_1448_team_0']
df.loc[518:, 'id_13_team_0'] = df.loc[518:, 'id_1402_team_0']

# CB2
df.loc[63:, 'id_10_team_0'] = df.loc[63:, 'id_99_team_0']
df.loc[69:, 'id_10_team_0'] = df.loc[69:, 'id_10_team_0']
df.loc[85:, 'id_10_team_0'] = df.loc[85:, 'id_70_team_1']
df.loc[88:, 'id_10_team_0'] = df.loc[88:, 'id_172_team_0']
df.loc[97:, 'id_10_team_0'] = df.loc[97:, 'id_197_team_0']
df.loc[100:, 'id_10_team_0'] = df.loc[100:, 'id_226_team_0']
df.loc[101:, 'id_10_team_0'] = df.loc[101:, 'id_235_team_0']
df.loc[103:, 'id_10_team_0'] = df.loc[103:, 'id_247_team_0']
df.loc[141:, 'id_10_team_0'] = df.loc[141:, 'id_387_team_0']
df.loc[154:, 'id_10_team_0'] = df.loc[154:, 'id_433_team_0']
df.loc[156:, 'id_10_team_0'] = df.loc[156:, 'id_437_team_0']
df.loc[175:, 'id_10_team_0'] = df.loc[175:, 'id_468_team_0']
df.loc[197:, 'id_10_team_0'] = df.loc[197:, 'id_503_team_0']
df.loc[224:, 'id_10_team_0'] = df.loc[224:, 'id_581_team_0']
df.loc[229:, 'id_10_team_0'] = df.loc[229:, 'id_604_team_0']
df.loc[297:, 'id_10_team_0'] = df.loc[297:, 'id_368_team_0']
df.loc[304:, 'id_10_team_0'] = df.loc[304:, 'id_604_team_0']
df.loc[378:, 'id_10_team_0'] = df.loc[378:, 'id_822_team_0']
df.loc[404:, 'id_10_team_0'] = df.loc[404:, 'id_876_team_0']
df.loc[405:, 'id_10_team_0'] = df.loc[405:, 'id_604_team_0']
df.loc[414:, 'id_10_team_0'] = df.loc[414:, 'id_931_team_0']
df.loc[427:, 'id_10_team_0'] = df.loc[427:, 'id_1063_team_0']
df.loc[461:, 'id_10_team_0'] = df.loc[461:, 'id_1263_team_0']
df.loc[466:, 'id_10_team_0'] = df.loc[466:, 'id_1243_team_1']
df.loc[473:, 'id_10_team_0'] = df.loc[473:, 'id_1346_team_0']
df.loc[479:, 'id_10_team_0'] = df.loc[479:, 'id_1135_team_1']
df.loc[481:, 'id_10_team_0'] = df.loc[481:, 'id_1378_team_0']
df.loc[483:, 'id_10_team_0'] = df.loc[483:, 'id_1220_team_1']
df.loc[508:, 'id_10_team_0'] = df.loc[508:, 'id_1346_team_0']
df.loc[510:, 'id_10_team_0'] = df.loc[510:, 'id_1341_team_0']

# CM1
df.loc[64:, 'id_2_team_0'] = df.loc[64:, 'id_108_team_0']
df.loc[100:, 'id_2_team_0'] = df.loc[100:, 'id_105_team_0']
df.loc[103:, 'id_2_team_0'] = df.loc[103:, 'id_256_team_0']
df.loc[133:, 'id_2_team_0'] = df.loc[133:, 'id_72_team_0']
df.loc[138:, 'id_2_team_0'] = df.loc[138:, 'id_367_team_0']
df.loc[148:, 'id_2_team_0'] = df.loc[148:, 'id_422_team_0']
df.loc[157:, 'id_2_team_0'] = df.loc[157:, 'id_392_team_1']
df.loc[160:, 'id_2_team_0'] = df.loc[160:, 'id_443_team_0']
df.loc[178:, 'id_2_team_0'] = df.loc[178:, 'id_474_team_0']
df.loc[180:, 'id_2_team_0'] = df.loc[180:, 'id_477_team_0']
df.loc[198:, 'id_2_team_0'] = df.loc[198:, 'id_468_team_0']
df.loc[200:, 'id_2_team_0'] = df.loc[200:, 'id_518_team_0']
df.loc[224:, 'id_2_team_0'] = df.loc[224:, 'id_580_team_0']
df.loc[226:, 'id_2_team_0'] = df.loc[226:, 'id_592_team_0']
df.loc[240:, 'id_2_team_0'] = df.loc[240:, 'id_581_team_0']
df.loc[242:, 'id_2_team_0'] = df.loc[242:, 'id_626_team_0']
df.loc[266:, 'id_2_team_0'] = df.loc[266:, 'id_657_team_0']
df.loc[271:, 'id_2_team_0'] = df.loc[271:, 'id_626_team_0']
df.loc[280:, 'id_2_team_0'] = df.loc[280:, 'id_657_team_0']
df.loc[283:, 'id_2_team_0'] = df.loc[283:, 'id_692_team_0']
df.loc[286:, 'id_2_team_0'] = df.loc[286:, 'id_700_team_0']
df.loc[292:, 'id_2_team_0'] = df.loc[292:, 'id_724_team_0']
df.loc[424:, 'id_2_team_0'] = df.loc[424:, 'id_1027_team_0']
df.loc[431:, 'id_2_team_0'] = df.loc[431:, 'id_1103_team_0']
df.loc[460:, 'id_2_team_0'] = df.loc[460:, 'id_1249_team_0']
df.loc[495:, 'id_2_team_0'] = df.loc[495:, 'id_1412_team_0']
df.loc[521:, 'id_2_team_0'] = df.loc[521:, 'id_1489_team_0']
df.loc[523:, 'id_2_team_0'] = df.loc[523:, 'id_1445_team_1']
df.loc[525:, 'id_2_team_0'] = df.loc[525:, 'id_1499_team_1']
df.loc[530:, 'id_2_team_0'] = df.loc[530:, 'id_1503_team_1']
df.loc[532:, 'id_2_team_0'] = df.loc[532:, 'id_1489_team_0']
df.loc[541:, 'id_2_team_0'] = df.loc[541:, 'id_1511_team_0']
df.loc[543:, 'id_2_team_0'] = df.loc[543:, 'id_1445_team_1']

# CM2
df.loc[55:, 'id_4_team_0'] = df.loc[55:, 'id_63_team_0']
df.loc[57:, 'id_4_team_0'] = df.loc[57:, 'id_66_team_0']
df.loc[78:, 'id_4_team_0'] = df.loc[78:, 'id_147_team_0']
df.loc[103:, 'id_4_team_0'] = df.loc[103:, 'id_264_team_0']
df.loc[106:, 'id_4_team_0'] = df.loc[106:, 'id_279_team_0']
df.loc[116:, 'id_4_team_0'] = df.loc[116:, 'id_264_team_0']
df.loc[122:, 'id_4_team_0'] = df.loc[122:, 'id_248_team_1']
df.loc[125:, 'id_4_team_0'] = df.loc[125:, 'id_306_team_0']
df.loc[223:, 'id_4_team_0'] = df.loc[223:, 'id_556_team_0']
df.loc[235:, 'id_4_team_0'] = df.loc[235:, 'id_393_team_0']
df.loc[236:, 'id_4_team_0'] = df.loc[236:, 'id_556_team_0']
df.loc[259:, 'id_4_team_0'] = df.loc[259:, 'id_544_team_1']
df.loc[262:, 'id_4_team_0'] = df.loc[262:, 'id_552_team_0']
df.loc[269:, 'id_4_team_0'] = df.loc[269:, 'id_671_team_0']
df.loc[304:, 'id_4_team_0'] = df.loc[304:, 'id_652_team_0']
df.loc[430:, 'id_4_team_0'] = df.loc[430:, 'id_1089_team_0']
df.loc[432:, 'id_4_team_0'] = df.loc[432:, 'id_582_team_1']
df.loc[434:, 'id_4_team_0'] = df.loc[434:, 'id_1116_team_0']
df.loc[459:, 'id_4_team_0'] = df.loc[459:, 'id_1238_team_0']
df.loc[534:, 'id_4_team_0'] = df.loc[534:, 'id_1507_team_0']

# CM3
df.loc[28:, 'id_16_team_0'] = df.loc[28:, 'id_37_team_0']
df.loc[55:, 'id_16_team_0'] = df.loc[55:, 'id_61_team_0']
df.loc[70:, 'id_16_team_0'] = df.loc[70:, 'id_131_team_0']
df.loc[76:, 'id_16_team_0'] = df.loc[76:, 'id_61_team_0']
df.loc[81:, 'id_16_team_0'] = df.loc[81:, 'id_160_team_0']
df.loc[89:, 'id_16_team_0'] = df.loc[89:, 'id_45_team_0']
df.loc[92:, 'id_16_team_0'] = df.loc[92:, 'id_183_team_0']
df.loc[103:, 'id_16_team_0'] = df.loc[103:, 'id_257_team_0']
df.loc[128:, 'id_16_team_0'] = df.loc[128:, 'id_276_team_1']
df.loc[138:, 'id_16_team_0'] = df.loc[138:, 'id_368_team_0']
df.loc[154:, 'id_16_team_0'] = df.loc[154:, 'id_434_team_0']
df.loc[164:, 'id_16_team_0'] = df.loc[164:, 'id_432_team_1']
df.loc[184:, 'id_16_team_0'] = df.loc[184:, 'id_486_team_0']
df.loc[186:, 'id_16_team_0'] = df.loc[186:, 'id_393_team_0']
df.loc[218:, 'id_16_team_0'] = df.loc[218:, 'id_432_team_1']
df.loc[222:, 'id_16_team_0'] = df.loc[222:, 'id_536_team_0']
df.loc[227:, 'id_16_team_0'] = df.loc[227:, 'id_597_team_0']
df.loc[266:, 'id_16_team_0'] = df.loc[266:, 'id_664_team_0']
df.loc[307:, 'id_16_team_0'] = df.loc[307:, 'id_760_team_0']
df.loc[402:, 'id_16_team_0'] = df.loc[402:, 'id_625_team_0']
df.loc[421:, 'id_16_team_0'] = df.loc[421:, 'id_984_team_0']
df.loc[423:, 'id_16_team_0'] = df.loc[423:, 'id_1011_team_0']
df.loc[430:, 'id_16_team_0'] = df.loc[430:, 'id_836_team_0']
df.loc[433:, 'id_16_team_0'] = df.loc[433:, 'id_1111_team_0']
df.loc[458:, 'id_16_team_0'] = df.loc[458:, 'id_1216_team_0']
df.loc[470:, 'id_16_team_0'] = df.loc[470:, 'id_1111_team_0']
df.loc[472:, 'id_16_team_0'] = df.loc[472:, 'id_1343_team_0']
df.loc[473:, 'id_16_team_0'] = df.loc[473:, 'id_1106_team_1']
df.loc[474:, 'id_16_team_0'] = df.loc[474:, 'id_1250_team_1']
df.loc[484:, 'id_16_team_0'] = df.loc[484:, 'id_1389_team_0']
df.loc[508:, 'id_16_team_0'] = df.loc[508:, 'id_1250_team_1']
df.loc[513:, 'id_16_team_0'] = df.loc[513:, 'id_1347_team_0']
df.loc[522:, 'id_16_team_0'] = df.loc[522:, 'id_1494_team_0']
df.loc[526:, 'id_16_team_0'] = df.loc[526:, 'id_1502_team_0']

# GK
df.loc[129:, 'id_281_team_0'] = df.loc[129:, 'id_268_team_1']
df.loc[132:, 'id_281_team_0'] = df.loc[132:, 'id_323_team_0']
df.loc[138:, 'id_281_team_0'] = df.loc[138:, 'id_281_team_0']
df.loc[151:, 'id_281_team_0'] = df.loc[151:, 'id_428_team_0']
df.loc[202:, 'id_281_team_0'] = df.loc[202:, 'id_524_team_0']
df.loc[233:, 'id_281_team_0'] = df.loc[233:, 'id_614_team_0']
df.loc[292:, 'id_281_team_0'] = df.loc[292:, 'id_725_team_0']
df.loc[428:, 'id_281_team_0'] = df.loc[428:, 'id_1074_team_1']

# LB
df.loc[69:, 'id_5_team_0'] = df.loc[69:, 'id_129_team_0']
df.loc[94:, 'id_5_team_0'] = df.loc[94:, 'id_188_team_0']
df.loc[105:, 'id_5_team_0'] = df.loc[105:, 'id_270_team_0']
df.loc[138:, 'id_5_team_0'] = df.loc[138:, 'id_365_team_0']
df.loc[196:, 'id_5_team_0'] = df.loc[196:, 'id_502_team_0']
df.loc[212:, 'id_5_team_0'] = df.loc[212:, 'id_537_team_0']
df.loc[232:, 'id_5_team_0'] = df.loc[232:, 'id_608_team_0']
df.loc[252:, 'id_5_team_0'] = df.loc[252:, 'id_639_team_0']
df.loc[265:, 'id_5_team_0'] = df.loc[265:, 'id_652_team_0']
df.loc[287:, 'id_5_team_0'] = df.loc[287:, 'id_652_team_0']
df.loc[310:, 'id_5_team_0'] = df.loc[310:, 'id_765_team_0']
df.loc[394:, 'id_5_team_0'] = df.loc[394:, 'id_855_team_0']
df.loc[431:, 'id_5_team_0'] = df.loc[431:, 'id_1104_team_0']
df.loc[439:, 'id_5_team_0'] = df.loc[439:, 'id_1027_team_0']
df.loc[452:, 'id_5_team_0'] = df.loc[452:, 'id_582_team_1']
df.loc[453:, 'id_5_team_0'] = df.loc[453:, 'id_1104_team_0']
df.loc[458:, 'id_5_team_0'] = df.loc[458:, 'id_1212_team_0']
df.loc[472:, 'id_5_team_0'] = df.loc[472:, 'id_1339_team_0']
df.loc[505:, 'id_5_team_0'] = df.loc[505:, 'id_1420_team_0']
df.loc[509:, 'id_5_team_0'] = df.loc[509:, 'id_1027_team_0']

# LW

df.loc[70:, 'id_1_team_0'] = df.loc[70:, 'id_132_team_0']
df.loc[72:, 'id_1_team_0'] = df.loc[72:, 'id_135_team_0']
df.loc[90:, 'id_1_team_0'] = df.loc[90:, 'id_176_team_0']
df.loc[103:, 'id_1_team_0'] = df.loc[103:, 'id_253_team_0']
df.loc[106:, 'id_1_team_0'] = df.loc[106:, 'id_275_team_0']
df.loc[129:, 'id_1_team_0'] = df.loc[129:, 'id_111_team_1']
df.loc[144:, 'id_1_team_0'] = df.loc[144:, 'id_404_team_0']
df.loc[219:, 'id_1_team_0'] = df.loc[219:, 'id_552_team_0']
df.loc[235:, 'id_1_team_0'] = df.loc[235:, 'id_596_team_1']
df.loc[251:, 'id_1_team_0'] = df.loc[251:, 'id_635_team_1']
df.loc[254:, 'id_1_team_0'] = df.loc[254:, 'id_544_team_1']
df.loc[257:, 'id_1_team_0'] = df.loc[257:, 'id_644_team_0']
df.loc[265:, 'id_1_team_0'] = df.loc[265:, 'id_596_team_1']
df.loc[282:, 'id_1_team_0'] = df.loc[282:, 'id_544_team_1']
df.loc[285:, 'id_1_team_0'] = df.loc[285:, 'id_656_team_1']
df.loc[298:, 'id_1_team_0'] = df.loc[298:, 'id_708_team_1']
df.loc[398:, 'id_1_team_0'] = df.loc[398:, 'id_859_team_0']
df.loc[431:, 'id_1_team_0'] = df.loc[431:, 'id_1101_team_0']
df.loc[466:, 'id_1_team_0'] = df.loc[466:, 'id_1306_team_0']

# RB
df.loc[69:, 'id_45_team_0'] = df.loc[69:, 'id_130_team_0']
df.loc[78:, 'id_45_team_0'] = df.loc[78:, 'id_146_team_0']
df.loc[94:, 'id_45_team_0'] = df.loc[94:, 'id_186_team_0']
df.loc[98:, 'id_45_team_0'] = df.loc[98:, 'id_131_team_0']
df.loc[99:, 'id_45_team_0'] = df.loc[99:, 'id_45_team_0']
df.loc[103:, 'id_45_team_0'] = df.loc[103:, 'id_254_team_0']
df.loc[142:, 'id_45_team_0'] = df.loc[142:, 'id_393_team_0']
df.loc[150:, 'id_45_team_0'] = df.loc[150:, 'id_368_team_0']
df.loc[224:, 'id_45_team_0'] = df.loc[224:, 'id_584_team_0']
df.loc[266:, 'id_45_team_0'] = df.loc[266:, 'id_661_team_0']
df.loc[296:, 'id_45_team_0'] = df.loc[296:, 'id_739_team_1']
df.loc[413:, 'id_45_team_0'] = df.loc[413:, 'id_822_team_0']
df.loc[415:, 'id_45_team_0'] = df.loc[415:, 'id_763_team_1']
df.loc[417:, 'id_45_team_0'] = df.loc[417:, 'id_876_team_0']
df.loc[419:, 'id_45_team_0'] = df.loc[419:, 'id_604_team_0']
df.loc[424:, 'id_45_team_0'] = df.loc[424:, 'id_978_team_1']
df.loc[425:, 'id_45_team_0'] = df.loc[425:, 'id_931_team_0']
df.loc[429:, 'id_45_team_0'] = df.loc[429:, 'id_1083_team_0']
df.loc[454:, 'id_45_team_0'] = df.loc[454:, 'id_1093_team_1']
df.loc[461:, 'id_45_team_0'] = df.loc[461:, 'id_1268_team_0']
df.loc[464:, 'id_45_team_0'] = df.loc[464:, 'id_1296_team_0']
df.loc[466:, 'id_45_team_0'] = df.loc[466:, 'id_1309_team_0']
df.loc[478:, 'id_45_team_0'] = df.loc[478:, 'id_1373_team_0']
df.loc[480:, 'id_45_team_0'] = df.loc[480:, 'id_1309_team_0']
df.loc[484:, 'id_45_team_0'] = df.loc[484:, 'id_1386_team_0']
df.loc[506:, 'id_45_team_0'] = df.loc[506:, 'id_1425_team_0']
df.loc[508:, 'id_45_team_0'] = df.loc[508:, 'id_1373_team_0']
df.loc[510:, 'id_45_team_0'] = df.loc[510:, 'id_1372_team_1']
df.loc[518:, 'id_45_team_0'] = df.loc[518:, 'id_1425_team_0']
df.loc[521:, 'id_45_team_0'] = df.loc[521:, 'id_1373_team_0']

# RW
df.loc[46:, 'id_20_team_0'] = df.loc[46:, 'id_47_team_0']
df.loc[84:, 'id_20_team_0'] = df.loc[84:, 'id_165_team_0']
df.loc[103:, 'id_20_team_0'] = df.loc[103:, 'id_258_team_0']
df.loc[108:, 'id_20_team_0'] = df.loc[108:, 'id_288_team_0']
df.loc[137:, 'id_20_team_0'] = df.loc[137:, 'id_357_team_0']
df.loc[166:, 'id_20_team_0'] = df.loc[166:, 'id_448_team_0']
df.loc[224:, 'id_20_team_0'] = df.loc[224:, 'id_577_team_0']
df.loc[266:, 'id_20_team_0'] = df.loc[266:, 'id_663_team_0']
df.loc[85:, 'id_20_team_0'] = df.loc[85:, 'id_696_team_0']
df.loc[287:, 'id_20_team_0'] = df.loc[287:, 'id_701_team_0']
df.loc[289:, 'id_20_team_0'] = df.loc[289:, 'id_709_team_0']
df.loc[293:, 'id_20_team_0'] = df.loc[293:, 'id_730_team_0']
df.loc[296:, 'id_20_team_0'] = df.loc[296:, 'id_741_team_0']
df.loc[298:, 'id_20_team_0'] = df.loc[298:, 'id_744_team_0']
df.loc[300:, 'id_20_team_0'] = df.loc[300:, 'id_750_team_0']
df.loc[431:, 'id_20_team_0'] = df.loc[431:, 'id_1098_team_0']
df.loc[459:, 'id_20_team_0'] = df.loc[459:, 'id_1234_team_0']
df.loc[478:, 'id_20_team_0'] = df.loc[478:, 'id_1368_team_0']
df.loc[541:, 'id_20_team_0'] = df.loc[541:, 'id_1452_team_1']

# ST
df.loc[66:, 'id_17_team_0'] = df.loc[66:, 'id_125_team_0']
df.loc[89:, 'id_17_team_0'] = df.loc[89:, 'id_17_team_0']
df.loc[92:, 'id_17_team_0'] = df.loc[92:, 'id_125_team_0']
df.loc[216:, 'id_17_team_0'] = df.loc[216:, 'id_543_team_0']
df.loc[222:, 'id_17_team_0'] = df.loc[222:, 'id_248_team_1']
df.loc[232:, 'id_17_team_0'] = df.loc[232:, 'id_607_team_0']
df.loc[428:, 'id_17_team_0'] = df.loc[428:, 'id_1073_team_0']
df.loc[466:, 'id_17_team_0'] = df.loc[466:, 'id_1310_team_0']
df.loc[468:, 'id_17_team_0'] = df.loc[468:, 'id_1320_team_0']
df.loc[480:, 'id_17_team_0'] = df.loc[480:, 'id_1376_team_0']
df.loc[482:, 'id_17_team_0'] = df.loc[482:, 'id_1383_team_0']
df.loc[487:, 'id_17_team_0'] = df.loc[487:, 'id_1310_team_0']
df.loc[523:, 'id_17_team_0'] = df.loc[523:, 'id_1495_team_0']

Team 1 - Liverpool

In [ ]:
#gk alisson: id22 lf51 ->id54 ltf57 ->id69 ltf62 ->id1099 ltf455 ->1174
df.loc[51:, 'id_22_team_1'] = df.loc[51:, 'id_54_team_1']
df.loc[57:, 'id_22_team_1'] = df.loc[57:, 'id_69_team_1']
df.loc[62:, 'id_22_team_1'] = df.loc[62:, 'id_1099_team_1']
df.loc[455:, 'id_22_team_1'] = df.loc[455:, 'id_1174_team_1']

In [ ]:
#cb1 vdj: id14 lf61-64 ->id121 ltf73-81 ->id163 ltf91-92 ->id187 ltf221-223 ->id576 ltf413-414 -> id936 ltf416-423 ->
#id658 ltf425-429 -> id1088 ltf453-460 ->id1266 ltf462-463 ->id1295 ltf475-477 ->id1371 ltf508-509 ->
# id1452 ltf511-516 -> id1480 ltf522 ->id1480
df.loc[64:, 'id_14_team_1'] = df.loc[64:, 'id_121_team_1']
df.loc[81:, 'id_14_team_1'] = df.loc[81:, 'id_163_team_1']
df.loc[92:, 'id_14_team_1'] = df.loc[92:, 'id_187_team_1']
df.loc[223:, 'id_14_team_1'] = df.loc[223:, 'id_576_team_1']
df.loc[414:, 'id_14_team_1'] = df.loc[414:, 'id_936_team_1']
df.loc[423:, 'id_14_team_1'] = df.loc[423:, 'id_658_team_1']
df.loc[429:, 'id_14_team_1'] = df.loc[429:, 'id_1088_team_1']
df.loc[460:, 'id_14_team_1'] = df.loc[460:, 'id_1266_team_1']
df.loc[463:, 'id_14_team_1'] = df.loc[463:, 'id_1295_team_1']
df.loc[477:, 'id_14_team_1'] = df.loc[477:, 'id_1371_team_1']
df.loc[509:, 'id_14_team_1'] = df.loc[509:, 'id_1452_team_1']
df.loc[516:, 'id_14_team_1'] = df.loc[516:, 'id_1480_team_1']
df.loc[522:, 'id_14_team_1'] = df.loc[522:, 'id_1480_team_1']

In [ ]:
#cb2 konate: id11 lf61 ->id84 lf63-64->id118 lf91-96->id185 lf142->id248->
#id125 f206->id248 lf222->id575->f231 id248 lf419-429->id1086 lf453-458->id1230 lf476->
#id1366 lf485->id1383 lf509-511->id1462 lf531-532->id1505
df.loc[61:, 'id_11_team_1'] = df.loc[61:, 'id_84_team_1']
df.loc[64:, 'id_11_team_1'] = df.loc[64:, 'id_118_team_1']
df.loc[96:, 'id_11_team_1'] = df.loc[96:, 'id_185_team_1']
df.loc[142:, 'id_11_team_1'] = df.loc[142:, 'id_248_team_1']
df.loc[143, 'id_11_team_1'] = df.loc[143, 'id_125_team_0']
df.loc[206:, 'id_11_team_1'] = df.loc[206:, 'id_248_team_1']
df.loc[222:, 'id_11_team_1'] = df.loc[222:, 'id_575_team_1']
df.loc[223, 'id_11_team_1'] = df.loc[223, 'id_248_team_1']
df.loc[429:, 'id_11_team_1'] = df.loc[429:, 'id_1086_team_1']
df.loc[458:, 'id_11_team_1'] = df.loc[458:, 'id_1230_team_1']
df.loc[476:, 'id_11_team_1'] = df.loc[476:, 'id_1366_team_1']
df.loc[485:, 'id_11_team_1'] = df.loc[485:, 'id_1383_team_0']
df.loc[511:, 'id_11_team_1'] = df.loc[511:, 'id_1462_team_1']
df.loc[532:, 'id_11_team_1'] = df.loc[532:, 'id_1505_team_1']

In [ ]:
#rb tt: id3 lf49-> id48 lf51-52->id57 lf63->id111 lf98->id206 lf218->id550 lf230-232->id613 lf255->id643 lf257->id622 lf344-345->
# id786 lf347-354->id798 lf387->id845 lf421->id999 f422->id607 lf425-428->id1077 lf480->id1381 lf527->id1504
df.loc[49:, 'id_3_team_1'] = df.loc[49:, 'id_48_team_1']
df.loc[52:, 'id_3_team_1'] = df.loc[52:, 'id_57_team_1']
df.loc[63:, 'id_3_team_1'] = df.loc[63:, 'id_111_team_1']
df.loc[98:, 'id_3_team_1'] = df.loc[98:, 'id_206_team_1']
df.loc[218:, 'id_3_team_1'] = df.loc[218:, 'id_550_team_1']
df.loc[232:, 'id_3_team_1'] = df.loc[232:, 'id_613_team_1']
df.loc[255:, 'id_3_team_1'] = df.loc[255:, 'id_643_team_1']
df.loc[257:, 'id_3_team_1'] = df.loc[257:, 'id_622_team_1']
df.loc[345:, 'id_3_team_1'] = df.loc[345:, 'id_786_team_1']
df.loc[354:, 'id_3_team_1'] = df.loc[354:, 'id_798_team_1']
df.loc[387:, 'id_3_team_1'] = df.loc[387:, 'id_845_team_1']
df.loc[421:, 'id_3_team_1'] = df.loc[421:, 'id_999_team_1']
df.loc[422:, 'id_3_team_1'] = df.loc[422:, 'id_607_team_0']
df.loc[428:, 'id_3_team_1'] = df.loc[428:, 'id_1077_team_1']
df.loc[480:, 'id_3_team_1'] = df.loc[480:, 'id_1381_team_1']
df.loc[527:, 'id_3_team_1'] = df.loc[527:, 'id_1504_team_1']

In [ ]:
#lb robo: id9 lf58->id71 lf61-65->id127 lf77-80->id162 lf86-90 ->id177 lf104-106->id284 lf132-147->id421 lf150-152->id432 lf159-167->
# id422 lf169-173->id465 f174 ->id393 lf178-187->id494 lf189->id495 f190->id474 191-204 ->id381 205-206 lf207 ->
# id534 208-213->id474 214 lf215->id541 216 lf217->id548 218-220->id381 f221->id503 f222 lf223-225->id581 lf240->id592 f241-242->
# id564 lf245->id585 lf248-251->id637 lf252->id641 lf265->id658 lf405-429->id1093 lf452-471->id1340 lf474-477->id1370
df.loc[58:, 'id_9_team_1'] = df.loc[58:, 'id_71_team_1']
df.loc[65:, 'id_9_team_1'] = df.loc[65:, 'id_127_team_1']
df.loc[80:, 'id_9_team_1'] = df.loc[80:, 'id_162_team_1']
df.loc[90:, 'id_9_team_1'] = df.loc[90:, 'id_177_team_1']
df.loc[106:, 'id_9_team_1'] = df.loc[106:, 'id_284_team_1']
df.loc[147:, 'id_9_team_1'] = df.loc[147:, 'id_421_team_1']
df.loc[152:, 'id_9_team_1'] = df.loc[152:, 'id_432_team_1']
df.loc[167:, 'id_9_team_1'] = df.loc[167:, 'id_422_team_0']
df.loc[173:, 'id_9_team_1'] = df.loc[173:, 'id_465_team_1']
df.loc[174:, 'id_9_team_1'] = df.loc[174:, 'id_393_team_0']
df.loc[187:, 'id_9_team_1'] = df.loc[187:, 'id_494_team_1']
df.loc[189:, 'id_9_team_1'] = df.loc[189:, 'id_495_team_1']
df.loc[190:, 'id_9_team_1'] = df.loc[190:, 'id_474_team_0']
df.loc[204:, 'id_9_team_1'] = df.loc[204:, 'id_381_team_0']
df.loc[207:, 'id_9_team_1'] = df.loc[207:, 'id_534_team_1']
df.loc[213:, 'id_9_team_1'] = df.loc[213:, 'id_474_team_0']
df.loc[215:, 'id_9_team_1'] = df.loc[215:, 'id_541_team_1']
df.loc[217:, 'id_9_team_1'] = df.loc[217:, 'id_548_team_1']
df.loc[220:, 'id_9_team_1'] = df.loc[220:, 'id_381_team_0']
df.loc[221:, 'id_9_team_1'] = df.loc[221:, 'id_503_team_0']
df.loc[225:, 'id_9_team_1'] = df.loc[225:, 'id_581_team_0']
df.loc[240:, 'id_9_team_1'] = df.loc[240:, 'id_592_team_0']
df.loc[242:, 'id_9_team_1'] = df.loc[242:, 'id_564_team_1']
df.loc[245:, 'id_9_team_1'] = df.loc[245:, 'id_585_team_1']
df.loc[251:, 'id_9_team_1'] = df.loc[251:, 'id_637_team_1']
df.loc[252:, 'id_9_team_1'] = df.loc[252:, 'id_641_team_1']
df.loc[265:, 'id_9_team_1'] = df.loc[265:, 'id_658_team_1']
df.loc[429:, 'id_9_team_1'] = df.loc[429:, 'id_1093_team_1']
df.loc[471:, 'id_9_team_1'] = df.loc[471:, 'id_1340_team_1']
df.loc[477:, 'id_9_team_1'] = df.loc[477:, 'id_1370_team_1']

In [ ]:
#cm grav: id6 0-40 -> id46 42-50 ->id53 52-> id101 63-85 -> id101 93-94 ->id160 95 ->id61 97->id47 100->id267 104->id276 106-111->
# id267 112-128->id315 130-134->id257 135-136->id276 137-153->id436 155-220->id582 224-428->id1106 432-452->
# id1198 457->id1250 460-470->id1198 471->id1344 473-483->id1390 485-489->id1249 493-504->id1445 509-522->id1503 526 528-529 532-544
df.loc[0:40, 'id_6_team_1'] = df.loc[0:40, 'id_6_team_1']
df.loc[42:50, 'id_6_team_1'] = df.loc[42:50, 'id_46_team_1']
df.loc[52, 'id_6_team_1'] = df.loc[52, 'id_53_team_1']
df.loc[63:85, 'id_6_team_1'] = df.loc[63:85, 'id_101_team_1']
df.loc[93:94, 'id_6_team_1'] = df.loc[93:94, 'id_101_team_1']
df.loc[95, 'id_6_team_1'] = df.loc[95, 'id_160_team_0']
df.loc[97, 'id_6_team_1'] = df.loc[97, 'id_61_team_0']
df.loc[100, 'id_6_team_1'] = df.loc[100, 'id_47_team_0']
df.loc[104, 'id_6_team_1'] = df.loc[104, 'id_267_team_1']
df.loc[106:111, 'id_6_team_1'] = df.loc[106:111, 'id_276_team_1']
df.loc[112:128, 'id_6_team_1'] = df.loc[112:128, 'id_267_team_1']
df.loc[130:134, 'id_6_team_1'] = df.loc[130:134, 'id_315_team_1']
df.loc[135:136, 'id_6_team_1'] = df.loc[135:136, 'id_257_team_0']
df.loc[137:153, 'id_6_team_1'] = df.loc[137:153, 'id_276_team_1']
df.loc[155:220, 'id_6_team_1'] = df.loc[155:220, 'id_436_team_1']
df.loc[224:428, 'id_6_team_1'] = df.loc[224:428, 'id_582_team_1']
df.loc[432:452, 'id_6_team_1'] = df.loc[432:452, 'id_1106_team_1']
df.loc[457, 'id_6_team_1'] = df.loc[457, 'id_1198_team_1']
df.loc[460:470, 'id_6_team_1'] = df.loc[460:470, 'id_1250_team_1']
df.loc[471, 'id_6_team_1'] = df.loc[471, 'id_1198_team_1']
df.loc[473:483, 'id_6_team_1'] = df.loc[473:483, 'id_1344_team_1']
df.loc[485:489, 'id_6_team_1'] = df.loc[485:489, 'id_1390_team_1']
df.loc[493:504, 'id_6_team_1'] = df.loc[493:504, 'id_1249_team_0']
df.loc[509:522, 'id_6_team_1'] = df.loc[509:522, 'id_1445_team_1']
df.loc[526, 'id_6_team_1'] = df.loc[526, 'id_1503_team_1']
df.loc[528:529, 'id_6_team_1'] = df.loc[528:529, 'id_1503_team_1']
df.loc[532:544, 'id_6_team_1'] = df.loc[532:544, 'id_1503_team_1']

In [ ]:
#cm mc: id12 0-9 11-14->id16 15-57->id81 61->id97 63-93->id147 98-99->id225 100->id234 101->id248 102->id264 110-115 122-123->id176 124->
# id248 125-131->id326 133-142->id402 144-146->id185 147->id402 148-215->id544 217-221->id578 224-244->id544 247-248->id635 250 252->
# id644 258->id648 262-273->id635 276-277->id650 278->id656 279->id639 280->id708 289-297->id753 301-376->id823 378-427->
# id1100 431-452->id582 455->id1027 456->id1104 458->id1337 472-475->id1369 478-531->id1506 533-534 537-541->id1238 542
df.loc[0:9, 'id_12_team_1'] = df.loc[0:9, 'id_12_team_1']
df.loc[11:14, 'id_12_team_1'] = df.loc[11:14, 'id_12_team_1']
df.loc[15:57, 'id_12_team_1'] = df.loc[15:57, 'id_16_team_0']
df.loc[61, 'id_12_team_1'] = df.loc[61, 'id_81_team_1']
df.loc[63:93, 'id_12_team_1'] = df.loc[63:93, 'id_97_team_1']
df.loc[98:99, 'id_12_team_1'] = df.loc[98:99, 'id_147_team_0']
df.loc[100, 'id_12_team_1'] = df.loc[100, 'id_225_team_1']
df.loc[101, 'id_12_team_1'] = df.loc[101, 'id_234_team_1']
df.loc[102, 'id_12_team_1'] = df.loc[102, 'id_248_team_1']
df.loc[110:115, 'id_12_team_1'] = df.loc[110:115, 'id_264_team_0']
df.loc[122:123, 'id_12_team_1'] = df.loc[122:123, 'id_264_team_0']
df.loc[124, 'id_12_team_1'] = df.loc[124, 'id_176_team_0']
df.loc[125:131, 'id_12_team_1'] = df.loc[125:131, 'id_248_team_1']
df.loc[133:142, 'id_12_team_1'] = df.loc[133:142, 'id_326_team_1']
df.loc[144:146, 'id_12_team_1'] = df.loc[144:146, 'id_402_team_1']
df.loc[147, 'id_12_team_1'] = df.loc[147, 'id_185_team_1']
df.loc[148:215, 'id_12_team_1'] = df.loc[148:215, 'id_402_team_1']
df.loc[217:221, 'id_12_team_1'] = df.loc[217:221, 'id_544_team_1']
df.loc[224:244, 'id_12_team_1'] = df.loc[224:244, 'id_578_team_1']
df.loc[247:248, 'id_12_team_1'] = df.loc[247:248, 'id_544_team_1']
df.loc[250, 'id_12_team_1'] = df.loc[250, 'id_635_team_1']
df.loc[258, 'id_12_team_1'] = df.loc[258, 'id_644_team_0']
df.loc[262:273, 'id_12_team_1'] = df.loc[262:273, 'id_648_team_1']
df.loc[276:277, 'id_12_team_1'] = df.loc[276:277, 'id_635_team_1']
df.loc[278, 'id_12_team_1'] = df.loc[278, 'id_650_team_1']
df.loc[279, 'id_12_team_1'] = df.loc[279, 'id_656_team_1']
df.loc[280, 'id_12_team_1'] = df.loc[280, 'id_639_team_0']
df.loc[289:297, 'id_12_team_1'] = df.loc[289:297, 'id_708_team_1']
df.loc[301:376, 'id_12_team_1'] = df.loc[301:376, 'id_753_team_1']
df.loc[378:427, 'id_12_team_1'] = df.loc[378:427, 'id_823_team_1']
df.loc[431:452, 'id_12_team_1'] = df.loc[431:452, 'id_1100_team_1']
df.loc[455, 'id_12_team_1'] = df.loc[455, 'id_582_team_1']
df.loc[456, 'id_12_team_1'] = df.loc[456, 'id_1027_team_0']
df.loc[458, 'id_12_team_1'] = df.loc[458, 'id_1104_team_0']
df.loc[472:475, 'id_12_team_1'] = df.loc[472:475, 'id_1337_team_1']
df.loc[478:531, 'id_12_team_1'] = df.loc[478:531, 'id_1369_team_1']
df.loc[533:534, 'id_12_team_1'] = df.loc[533:534, 'id_1506_team_1']
df.loc[537:541, 'id_12_team_1'] = df.loc[537:541, 'id_1506_team_1']
df.loc[542, 'id_12_team_1'] = df.loc[542, 'id_1238_team_0']

In [ ]:
#cam szobo: id8 0-60->id106 64-92->id252 103-206->id502 210-214->id539 217-219->id402 225->id596 227-234->id622 237-241 244-245->id596 246-260->id650 264->
# id656 265->id667 267-346->id789 348-410->id859 412-414->id945 416-428->id1090 430-454->id1197 457->id1350 474-544
df.loc[64:92, 'id_8_team_1'] = df.loc[64:92, 'id_106_team_1']
df.loc[103:206, 'id_8_team_1'] = df.loc[103:206, 'id_252_team_1']
df.loc[210:214, 'id_8_team_1'] = df.loc[210:214, 'id_502_team_0']
df.loc[217:219, 'id_8_team_1'] = df.loc[217:219, 'id_539_team_0']
df.loc[225, 'id_8_team_1'] = df.loc[225, 'id_402_team_1']
df.loc[227:234, 'id_8_team_1'] = df.loc[227:234, 'id_596_team_1']
df.loc[237:241, 'id_8_team_1'] = df.loc[237:241, 'id_622_team_1']
df.loc[244:245, 'id_8_team_1'] = df.loc[244:245, 'id_622_team_1']
df.loc[246:260, 'id_8_team_1'] = df.loc[246:260, 'id_596_team_1']
df.loc[264, 'id_8_team_1'] = df.loc[264, 'id_650_team_1']
df.loc[265, 'id_8_team_1'] = df.loc[265, 'id_656_team_1']
df.loc[267:436, 'id_8_team_1'] = df.loc[267:436, 'id_667_team_1']
df.loc[348:410, 'id_8_team_1'] = df.loc[348:410, 'id_789_team_1']
df.loc[412:414, 'id_8_team_1'] = df.loc[412:414, 'id_859_team_0']
df.loc[416:428, 'id_8_team_1'] = df.loc[416:428, 'id_945_team_1']
df.loc[430:454, 'id_8_team_1'] = df.loc[430:454, 'id_1090_team_1']
df.loc[457, 'id_8_team_1'] = df.loc[457, 'id_1197_team_1']
df.loc[474:544, 'id_8_team_1'] = df.loc[474:544, 'id_1350_team_1']

In [ ]:
#lw diaz: id15 0-56->id70 59->id124 66-82->id167 85->id10 86->id70 87-89->id268 104-120->id172 121-126->id322 132-165->id367 166->id496 191-219->id598 227-263->
# id662 266-401->id883 406-418->id1095 430-449->id1138 451-452->id1315 467-475->id1372 478-506->id1446 509-519->id1490 521-544
df.loc[59, 'id_15_team_1'] = df.loc[59, 'id_70_team_1']
df.loc[66:82, 'id_15_team_1'] = df.loc[66:82, 'id_124_team_1']
df.loc[85, 'id_15_team_1'] = df.loc[85, 'id_167_team_1']
df.loc[86, 'id_15_team_1'] = df.loc[86, 'id_10_team_0']
df.loc[87:89, 'id_15_team_1'] = df.loc[87:89, 'id_70_team_1']
df.loc[104:120, 'id_15_team_1'] = df.loc[104:120, 'id_268_team_1']
df.loc[121:126, 'id_15_team_1'] = df.loc[121:126, 'id_172_team_0']
df.loc[132:165, 'id_15_team_1'] = df.loc[132:165, 'id_322_team_1']
df.loc[166, 'id_15_team_1'] = df.loc[166, 'id_367_team_0']
df.loc[191:219, 'id_15_team_1'] = df.loc[191:219, 'id_496_team_1']
df.loc[227:263, 'id_15_team_1'] = df.loc[227:263, 'id_598_team_1']
df.loc[266:401, 'id_15_team_1'] = df.loc[266:401, 'id_662_team_1']
df.loc[406:418, 'id_15_team_1'] = df.loc[406:418, 'id_883_team_1']
df.loc[430:449, 'id_15_team_1'] = df.loc[430:449, 'id_1095_team_1']
df.loc[451:452, 'id_15_team_1'] = df.loc[451:452, 'id_1138_team_1']
df.loc[467:475, 'id_15_team_1'] = df.loc[467:475, 'id_1315_team_1']
df.loc[478:506, 'id_15_team_1'] = df.loc[478:506, 'id_1372_team_1']
df.loc[509:519, 'id_15_team_1'] = df.loc[509:519, 'id_1446_team_1']
df.loc[521:544, 'id_15_team_1'] = df.loc[521:544, 'id_1490_team_1']

In [ ]:
#rw salah: id7 0-92->id191 95-103->id269 105-134->id363 138-206 208-209->id535 211-221->id404 222->id590 225-264->id668 267-278->id687 280-288->id720 291->
# id728 293-309->id656 310-335 338-375->id809 376-380->id841 386-400->id869 403->id724 406->id1060 427-453->id1100 454->id582 456->id1027 458->
# id1104 460-461->id1307 466-472->id1348 474-487->id1398 489-518->id1485 520-524 527-544
df.loc[95:103, 'id_7_team_1'] = df.loc[95:103, 'id_191_team_1']
df.loc[105:134, 'id_7_team_1'] = df.loc[105:134, 'id_269_team_1']
df.loc[138:206, 'id_7_team_1'] = df.loc[138:206, 'id_363_team_1']
df.loc[208:209, 'id_7_team_1'] = df.loc[208:209, 'id_363_team_1']
df.loc[211:221, 'id_7_team_1'] = df.loc[211:221, 'id_535_team_1']
df.loc[222, 'id_7_team_1'] = df.loc[222, 'id_404_team_0']
df.loc[225:264, 'id_7_team_1'] = df.loc[225:264, 'id_590_team_1']
df.loc[267:278, 'id_7_team_1'] = df.loc[267:278, 'id_668_team_1']
df.loc[280:288, 'id_7_team_1'] = df.loc[280:288, 'id_687_team_1']
df.loc[291, 'id_7_team_1'] = df.loc[291, 'id_720_team_1']
df.loc[293:309, 'id_7_team_1'] = df.loc[293:309, 'id_728_team_1']
df.loc[310:335, 'id_7_team_1'] = df.loc[310:335, 'id_656_team_1']
df.loc[338:375, 'id_7_team_1'] = df.loc[338:375, 'id_809_team_0']
df.loc[386:400, 'id_7_team_1'] = df.loc[386:400, 'id_841_team_1']
df.loc[403, 'id_7_team_1'] = df.loc[403, 'id_869_team_1']
df.loc[406, 'id_7_team_1'] = df.loc[406, 'id_724_team_0']
df.loc[427:453, 'id_7_team_1'] = df.loc[427:453, 'id_1060_team_1']
df.loc[454, 'id_7_team_1'] = df.loc[454, 'id_1100_team_1']
df.loc[456, 'id_7_team_1'] = df.loc[456, 'id_582_team_1']
df.loc[458, 'id_7_team_1'] = df.loc[458, 'id_1027_team_0']
df.loc[460:461, 'id_7_team_1'] = df.loc[460:461, 'id_1104_team_0']
df.loc[466:472, 'id_7_team_1'] = df.loc[466:472, 'id_1307_team_1']
df.loc[474:487, 'id_7_team_1'] = df.loc[474:487, 'id_1348_team_1']
df.loc[489:518, 'id_7_team_1'] = df.loc[489:518, 'id_1398_team_1']
df.loc[520:524, 'id_7_team_1'] = df.loc[520:524, 'id_1485_team_1']
df.loc[527:544, 'id_7_team_1'] = df.loc[527:544, 'id_1485_team_1']

In [ ]:
#st jota: id18 0-57->id126 66-96->id72 103-111->id105 112-140->id392 142-155->id438 157-177->id433 184-196->id532 206-213 217->id368 221->id564 222->id585 224-230->
# id368 231-271->id676 274-283 285-293->id739 295 ->id746 298-305->id763 309-314 317-324 327-329 334-408->id822 409->id876 411->id604 413->id931 419->id978 420->
# id1081 429-447->id931 448->id1135 450-452->id1143 453->id1157 454->id1191 456->id1207 457->id1220 458->id1243 459-461->
# id1263 462-475->id1367 478-488->id1341 490->id1349 495-496->id1392 497-503->id1458 511->id1389 513-514->id1479 517-544
df.loc[66:96, 'id_18_team_1'] = df.loc[66:96, 'id_126_team_1']
df.loc[103:111, 'id_18_team_1'] = df.loc[103:111, 'id_72_team_0']
df.loc[112:140, 'id_18_team_1'] = df.loc[112:140, 'id_105_team_0']
df.loc[142:155, 'id_18_team_1'] = df.loc[142:155, 'id_392_team_1']
df.loc[157:177, 'id_18_team_1'] = df.loc[157:177, 'id_438_team_1']
df.loc[184:196, 'id_18_team_1'] = df.loc[184:196, 'id_433_team_0']
df.loc[206:213, 'id_18_team_1'] = df.loc[206:213, 'id_532_team_1']
df.loc[217, 'id_18_team_1'] = df.loc[217, 'id_532_team_1']
df.loc[221, 'id_18_team_1'] = df.loc[221, 'id_368_team_0']
df.loc[222, 'id_18_team_1'] = df.loc[222, 'id_564_team_1']
df.loc[224:230, 'id_18_team_1'] = df.loc[224:230, 'id_585_team_1']
df.loc[231:271, 'id_18_team_1'] = df.loc[231:271, 'id_368_team_0']
df.loc[274:283, 'id_18_team_1'] = df.loc[274:283, 'id_676_team_1']
df.loc[285:293, 'id_18_team_1'] = df.loc[285:293, 'id_676_team_1']
df.loc[295, 'id_18_team_1'] = df.loc[295, 'id_739_team_1']
df.loc[298:305, 'id_18_team_1'] = df.loc[298:305, 'id_746_team_1']
df.loc[309:314, 'id_18_team_1'] = df.loc[309:314, 'id_763_team_1']
df.loc[317:324, 'id_18_team_1'] = df.loc[317:324, 'id_763_team_1']
df.loc[327:329, 'id_18_team_1'] = df.loc[327:329, 'id_763_team_1']
df.loc[334:408, 'id_18_team_1'] = df.loc[334:408, 'id_763_team_1']
df.loc[409, 'id_18_team_1'] = df.loc[409, 'id_822_team_0']
df.loc[411, 'id_18_team_1'] = df.loc[411, 'id_876_team_0']
df.loc[413, 'id_18_team_1'] = df.loc[413, 'id_604_team_0']
df.loc[419, 'id_18_team_1'] = df.loc[419, 'id_931_team_0']
df.loc[420, 'id_18_team_1'] = df.loc[420, 'id_978_team_1']
df.loc[429:447, 'id_18_team_1'] = df.loc[429:447, 'id_1081_team_1']
df.loc[448, 'id_18_team_1'] = df.loc[448, 'id_931_team_0']
df.loc[450:452, 'id_18_team_1'] = df.loc[450:452, 'id_1135_team_1']
df.loc[453, 'id_18_team_1'] = df.loc[453, 'id_1143_team_1']
df.loc[454, 'id_18_team_1'] = df.loc[454, 'id_1157_team_1']
df.loc[456, 'id_18_team_1'] = df.loc[456, 'id_1191_team_1']
df.loc[457, 'id_18_team_1'] = df.loc[457, 'id_1207_team_1']
df.loc[458, 'id_18_team_1'] = df.loc[458, 'id_1220_team_1']
df.loc[459:461, 'id_18_team_1'] = df.loc[459:461, 'id_1243_team_1']
df.loc[462:475, 'id_18_team_1'] = df.loc[462:475, 'id_1263_team_0']
df.loc[478:488, 'id_18_team_1'] = df.loc[478:488, 'id_1367_team_1']
df.loc[490, 'id_18_team_1'] = df.loc[490, 'id_1341_team_0']
df.loc[495:496, 'id_18_team_1'] = df.loc[495:496, 'id_1349_team_0']
df.loc[497:503, 'id_18_team_1'] = df.loc[497:503, 'id_1392_team_0']
df.loc[511, 'id_18_team_1'] = df.loc[511, 'id_1458_team_1']
df.loc[513:514, 'id_18_team_1'] = df.loc[513:514, 'id_1389_team_0']
df.loc[517:544, 'id_18_team_1'] = df.loc[517:544, 'id_1479_team_1']

In [ ]:
#save player ids and ball to new dataset
ids = ["frame_index",
    "id_13_team_0", "id_10_team_0", "id_2_team_0", "id_4_team_0",
    "id_16_team_0", "id_281_team_0", "id_5_team_0", "id_1_team_0",
    "id_45_team_0", "id_20_team_0", "id_17_team_0",
    "id_22_team_1", "id_14_team_1", "id_11_team_1", "id_3_team_1",
    "id_9_team_1", "id_6_team_1", "id_12_team_1", "id_8_team_1",
    "id_15_team_1", "id_7_team_1", "id_18_team_1",
    "ball"
]

df_ids = df[ids]  # This selects columns, not rows

# Save to CSV
df_ids.to_csv('../../data/raw-coords/train.csv', index=False)

Interpolation of missing values

In [ ]:
original_df = pd.read_csv('../../data/raw-coords/train.csv')

df = original_df.copy()

In [ ]:
import pandas as pd

# Helper function to split "x,y" strings into two float columns
def split_xy_columns(df, exclude_columns=None):
    new_df = pd.DataFrame(index=df.index)  # Preserva o índice original
    exclude_columns = exclude_columns or []

    for col in df.columns:
        if col in exclude_columns:
            new_df[col] = df[col]  # Copia a coluna sem alteração
            continue

        if df[col].dropna().empty:
            new_df[f"{col}_x"] = df[col]
            new_df[f"{col}_y"] = df[col]
        else:
            col_as_str = df[col].astype(str)
            xy_split = col_as_str.str.split(",", expand=True)
            new_df[f"{col}_x"] = pd.to_numeric(xy_split[0], errors='coerce')
            if 1 in xy_split.columns:
                new_df[f"{col}_y"] = pd.to_numeric(xy_split[1], errors='coerce')
            else:
                new_df[f"{col}_y"] = pd.NA
    return new_df

# Coluna que você NÃO quer separar
excluded = ['frame_index']  # ou o nome real da sua coluna de índice

# Step 1: Split into x and y (apenas jogadores e bola)
df_xy = split_xy_columns(df, exclude_columns=excluded)

# Step 2: Interpolar
df_xy_interpolated = df_xy.interpolate(method='linear', limit_direction='both')

# Step 3: Recombinar em "x,y" string format
df_combined = pd.DataFrame(index=df.index)
for col in df.columns:
    if col in excluded:
        df_combined[col] = df[col]  # copia sem alteração
    else:
        x_col = f"{col}_x"
        y_col = f"{col}_y"
        df_combined[col] = (
            df_xy_interpolated[x_col].round(2).astype(str) + "," +
            df_xy_interpolated[y_col].round(2).astype(str)
        )


Save Dataset

In [ ]:
#save new dataset to train folder
interpolated_xy_path = "../../data/train/train_interpolated_xy.csv"
interpolated_combined_path = "../../data/train/train_interpolated_combined.csv"
df_xy_interpolated.to_csv(interpolated_xy_path, index=False)
df_combined.to_csv(interpolated_combined_path, index=False)

interpolated_xy_path, interpolated_combined_path